In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from ganime.data.base import load_dataset
from ganime.model.p2p.p2p_v2 import P2P
import tensorflow as tf
from tqdm import tqdm
tf.get_logger().setLevel('ERROR')

In [4]:
strategy = tf.distribute.MirroredStrategy()

2022-05-02 01:19:06.923887: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 01:19:07.317115: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14252 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:25:00.0, compute capability: 8.6


In [5]:
train_ds, test_ds, input_shape = load_dataset("moving_mnist", "../../data", batch_size=100)# * strategy.num_replicas_in_sync)

In [6]:
train_ds_iterator = train_ds.as_numpy_iterator()
x = next(train_ds_iterator)

In [12]:
with strategy.scope():
    model = P2P()

In [13]:
model.encoder.summary(line_length=200)

Model: "encoder"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                                             Output Shape                                                                    Param #                       
 input_9 (InputLayer)                                                                     [(None, 1, 64, 64, 1)]                                                          0                             
                                                                                                                                                                                                        
 time_distributed_19 (TimeDistributed)                                                    (None, 1, 32, 32, 64)                                                           1088     

In [14]:
model.decoder.summary(line_length=200)

Model: "decoder"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                                             Output Shape                                                                    Param #                       
 input_10 (InputLayer)                                                                    [(None, 1, 128)]                                                                0                             
                                                                                                                                                                                                        
 dense_19 (Dense)                                                                         (None, 1, 512)                                                                  66048    

In [15]:
model.frame_predictor.summary(line_length=200)

Model: "frame_predictor"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                                             Output Shape                                                                    Param #                       
 input_6 (InputLayer)                                                                     [(None, None, 138)]                                                             0                             
                                                                                                                                                                                                        
 time_distributed_16 (TimeDistributed)                                                    (None, None, 256)                                                               3

In [16]:
with strategy.scope():
    model.compile(optimizer=tf.keras.optimizers.Adam(
            learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8
        ))
history = model.fit(train_ds, epochs=100)

2022-04-29 08:53:06.433202: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_UINT8
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 9000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 20
        }
        dim {
          size: 64
        }
        dim {
          size: 64
        }
        dim {
          size: 1
        }
      }
    }
  }
}



Epoch 1/100


  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

2022-04-29 08:54:35.089400: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:812] function_optimizer failed: INVALID_ARGUMENT: Input 0 of node zeros_like_2854 was passed float from model_2/lstm_4/PartitionedCall:7 incompatible with expected variant.
2022-04-29 08:54:36.209676: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:812] shape_optimizer failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 30
2022-04-29 08:54:36.786826: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:812] layout failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 30
2022-04-29 08:54:37.090430: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:812] remapper failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 30
2022-04-29 08:54:38.535141: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:812] tfg_optimizer{} failed: INVALID_ARGUMENT: Input 0 of node zeros_like_2854 was passed float from model_2/lstm_4/PartitionedCall:7 incompatible with expected v

90/90 [==============================] - 154s 506ms/step - loss: 31566.5317 - reconstruction_loss: 3695.5491 - kl_loss: 16.4917 - align_loss: 1131.9320 - cpc_loss: 263.4318
Epoch 2/100
90/90 [==============================] - 46s 509ms/step - loss: 20704.1126 - reconstruction_loss: 3310.6875 - kl_loss: 1.2351 - align_loss: 0.6221 - cpc_loss: 173.9940
Epoch 3/100
90/90 [==============================] - 46s 513ms/step - loss: 20710.4778 - reconstruction_loss: 3310.6873 - kl_loss: 0.9217 - align_loss: 0.2974 - cpc_loss: 174.0508
Epoch 4/100
90/90 [==============================] - 46s 514ms/step - loss: 20737.4548 - reconstruction_loss: 3310.6873 - kl_loss: 0.8092 - align_loss: 0.2161 - cpc_loss: 174.2491
Epoch 5/100
90/90 [==============================] - 47s 514ms/step - loss: 20783.7847 - reconstruction_loss: 3310.6875 - kl_loss: 0.7952 - align_loss: 0.1684 - cpc_loss: 174.6586
Epoch 6/100
90/90 [==============================] - 46s 514ms/step - loss: 20809.3614 - reconstruction_los


KeyboardInterrupt



In [9]:
model = tf.saved_model.load("./p2p_v2")

In [10]:
from ganime.visualization.videos import display_videos
generated = model(x[0])

2022-05-02 01:20:41.803740: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-02 01:20:42.830252: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [11]:
display_videos(generated, n_rows=1, n_cols=5)

In [12]:
display_videos(x[1], n_rows=1, n_cols=5)

In [20]:
generated

<tf.Tensor: shape=(100, 20, 64, 64, 1), dtype=float32, numpy=
array([[[[[0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00],
          ...,
          [0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00]],

         [[0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00],
          ...,
          [0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00]],

         [[0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00],
          ...,
          [0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00]],

         ...,

         [[0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00],
          ...,
          [0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00]],

         [[0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00],
          ...,
          [0.00000000e+00],
          [0.00000000e+00],
          [0.00000000e+00]],